<a href="https://colab.research.google.com/github/uol-mediaprocessing-202021/medienverarbeitung-b-color-and-tone-correction/blob/develop/ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

After trying to improve images with color and white balancing we decided to give machine learning a try. We decided to use a pre trained model from https://github.com/aiff22/DPED (paper: https://arxiv.org/pdf/1704.02470.pdf, project webpage: http://people.ee.ethz.ch/~ihnatova/index.html). In this paper the main objective is to improve pictures from smartphones. This is done by 'comparison' to the same picture taken with a proper DSLR (canon). We used Smartphone pictures and DSLR Pictures provided by the PyNET project (http://people.ee.ethz.ch/~ihnatova/pynet.html). This dataset is from the same institution (ETH Zürich), but uses a more modern smartphone.
The CNN they provided is a so called GAN (Generative Adversarial Network). A GAN consists of 2 different Networks. They basically work "against" each other. On a general scale the generator tries to generate something that appears to be real. On the other side the discriminator tries to discern whether the generated thing is real or generated by the generator. In our case that means the generator tries to generate a DSLR-equivalent image from the smartphone image. The discriminator tries to discern whether the image is generated by the generator or taken by an actual DSLR. The discriminator is basically a classifier, whilst the actual image is generated by the generator.

source: https://developers.google.com/machine-learning/gan/gan_structure 

The GAN we used differs slightly from the standard case shown in the graphic above: we do not use a random input. Our input is the smartphone picture. Often a GAN is used to create random images that appear to be real, e.g. Cats (e.g. https://thiscatdoesnotexist.com/ or People (e.g. https://thispersondoesnotexist.com/) generated from random input. However we want to improve a specific image up to DSLR Quality and not create a random one from scratch, so a GAN with a random input would not work in our case.

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys

accessing files on Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


navigate to the folder

In [ ]:
%cd /content/gdrive/MyDrive/DPED-master/DPED-master/

/content/gdrive/MyDrive/DPED-master/DPED-master


In [ ]:
!python load_dataset.py model=iphone

Here the training begins. We replaced the "iphone" training pictures with the images taken by the Huawei smartphone from the PyNet Dataset. The generator aswell as the discriminator get trained in this training process.

In [ ]:
!python train_model.py model=iphone train_size=10019

2021-01-31 15:32:35.357095: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term

The following parameters will be applied for CNN training:

Phone model: iphone
Batch size: 50
Learning rate: 0.0005
Training iterations: 20000

Content loss: 10
Color loss: 0.5
Texture loss: 1
Total variation loss: 2000

Path to DPED dataset: dped/
Path to VGG-19 network: vgg_pretrained/imagenet-vgg-verydeep-19.mat
Evaluation step: 1000

2021-01-31 15:32:37.456443: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-31 15:32:37.457371: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-31 15:32:37.478779: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), 

After training we can create some results by putting images into the test folder for "iphone" pictures. We included pictures from the original Dataset aswell as pictures taken with our own smartphones. Afterwards the results are accessible in the "visual_results" folder.

In [ ]:
!python test_model.py model=iphone iteration=18000 test_subset=full use_gpu=true

2021-02-05 17:39:27.618024: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term
2021-02-05 17:39:30.318950: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-05 17:39:30.320182: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-05 17:39:30.372824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-05 17:39:30.373396: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/